In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
# 変数urlにSUUMOホームページのURLを格納
url = "https://suumo.jp/chintai/tokyo/sc_shinjuku/?page={}"

# アクセスするためのURLをtarget_urlに格納する
target_url = url.format(1)

print(target_url)

# target_urlへのアクセス結果を変数rに格納
r = requests.get(target_url)

# 取得結果を解析してBeautifulSoupに格納
soup = BeautifulSoup(r.text)

contents = soup.find_all('div', class_='cassetteitem')

https://suumo.jp/chintai/tokyo/sc_shinjuku/?page=1


In [3]:
content = contents[0]

In [4]:
detail = content.find('div', class_='cassetteitem-detail')
table = content.find('table', class_='cassetteitem_other')

In [5]:
title = detail.find('div', class_='cassetteitem_content-title').text.replace('\n','').replace('\r','').replace('\t','').replace('\u3000','')
address = detail.find('li', class_='cassetteitem_detail-col1').text.replace('\n','').replace('\r','').replace('\t','').replace('\u3000','')
access = detail.find('li', class_='cassetteitem_detail-col2').text.replace('\n','').replace('\r','').replace('\t','').replace('\u3000','')
age = detail.find('li', class_='cassetteitem_detail-col3').text.replace('\n','').replace('\r','').replace('\t','').replace('\u3000',''),

In [6]:
tr_tags = table.find_all('tr', class_='js-cassette_link')
tr_tag = tr_tags[0]

In [7]:
floor, price, first_fee, capacity = tr_tag.find_all('td')[2:6]

In [10]:
capacity

<td>
<ul>
<li><span class="cassetteitem_madori">1DK</span></li>
<li><span class="cassetteitem_menseki">29.08m<sup>2</sup></span></li>
</ul>
</td>

In [12]:
fee, management_fee = price.find_all('li')
deposit, gratuity = first_fee.find_all('li')
madori, menseki = capacity.find_all('li')

In [16]:
d =  {
        'title': title,
        'address': address,
        'access': access,
        'age': age,
        'floor': floor.text.replace('\n','').replace('\r','').replace('\t','').replace('\u3000',''),
        'fee': fee.text,
        'management_fee': management_fee.text,
        'deposit': deposit.text,
        'gratuity': gratuity.text,
        'madori': madori.text,
        'menseki': menseki.text
        }

In [17]:
d

{'title': 'ジニア新宿中落合',
 'address': '東京都新宿区中落合２',
 'access': '西武新宿線/中井駅 歩9分西武新宿線/下落合駅 歩6分ＪＲ山手線/高田馬場駅 歩18分',
 'age': ('新築地下1地上3階建',),
 'floor': '1階',
 'fee': '11.7万円',
 'management_fee': '10000円',
 'deposit': '11.7万円',
 'gratuity': '-',
 'madori': '1DK',
 'menseki': '29.08m2'}

In [18]:
page = soup.select_one('ol.pagination-parts li:last-child a').text
page_max = int(page)

d_list = []
contents = soup.find_all('div', class_='cassetteitem')

for page_number in range(1, page_max):
    page_url = url.format(page_number)
    page_response = requests.get(page_url)

    for content in contents:
        detail = content.find('div', class_='cassetteitem-detail')
        table = content.find('table', class_='cassetteitem_other')

        title = detail.find('div', class_='cassetteitem_content-title').text.replace('\n','').replace('\r','').replace('\t','').replace('\u3000','')
        address = detail.find('li', class_='cassetteitem_detail-col1').text.replace('\n','').replace('\r','').replace('\t','').replace('\u3000','')
        access = detail.find('li', class_='cassetteitem_detail-col2').text.replace('\n','').replace('\r','').replace('\t','').replace('\u3000','')
        age = detail.find('li', class_='cassetteitem_detail-col3').text.replace('\n','').replace('\r','').replace('\t','').replace('\u3000',''),

        tr_tags = table.find_all('tr', class_='js-cassette_link')
        
        for tr_tag in tr_tags:
            floor, price, first_fee, capacity = tr_tag.find_all('td')[2:6]
            fee, management_fee = price.find_all('li')
            deposit, gratuity = first_fee.find_all('li')
            madori, menseki = capacity.find_all('li')
            
            d =  {
                'title': title,
                'address': address,
                'access': access,
                'age': age,
                'floor': floor.text.replace('\n','').replace('\r','').replace('\t','').replace('\u3000',''),
                'fee': fee.text,
                'management_fee': management_fee.text,
                'deposit': deposit.text,
                'gratuity': gratuity.text,
                'madori': madori.text,
                'menseki': menseki.text
            }

            d_list.append(d)

In [19]:
# pandasでデータフレームに入れて、csvで出力
import pandas as pd
df = pd.DataFrame(d_list)

In [20]:
df

,title,address,access,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
0,ジニア新宿中落合,東京都新宿区中落合２,西武新宿線/中井駅 歩9分西武新宿線/下落合駅 歩6分ＪＲ山手線/高田馬場駅 歩18分,"(新築地下1地上3階建,)",1階,11.7万円,10000円,11.7万円,-,1DK,29.08m2
1,ジニア新宿中落合,東京都新宿区中落合２,西武新宿線/中井駅 歩9分西武新宿線/下落合駅 歩6分ＪＲ山手線/高田馬場駅 歩18分,"(新築地下1地上3階建,)",1階,16.3万円,10000円,16.3万円,-,1LDK,39.97m2
2,ジニア新宿中落合,東京都新宿区中落合２,西武新宿線/中井駅 歩9分西武新宿線/下落合駅 歩6分ＪＲ山手線/高田馬場駅 歩18分,"(新築地下1地上3階建,)",B1階,16.3万円,10000円,16.3万円,-,1LDK,39.97m2
3,ジニア新宿中落合,東京都新宿区中落合２,西武新宿線/中井駅 歩9分西武新宿線/下落合駅 歩6分ＪＲ山手線/高田馬場駅 歩18分,"(新築地下1地上3階建,)",B1階,16.3万円,10000円,16.3万円,-,1LDK,39.97m2
4,ジニア新宿中落合,東京都新宿区中落合２,西武新宿線/中井駅 歩9分西武新宿線/下落合駅 歩6分ＪＲ山手線/高田馬場駅 歩18分,"(新築地下1地上3階建,)",B1階,16.5万円,10000円,16.5万円,-,1LDK,39.97m2
...,...,...,...,...,...,...,...,...,...,...,...
26112,ライオンズマンション高田馬場第２,東京都新宿区高田馬場４,ＪＲ山手線/高田馬場駅 歩6分西武新宿線/下落合駅 歩15分東京メトロ副都心線/西早稲田駅 ...,"(築27年地下1地上5階建,)",3階,19万円,5000円,19万円,19万円,1SLDK,64.91m2
26113,パークアクシス市谷左内町,東京都新宿区市谷左内町,都営新宿線/市ケ谷駅 歩6分ＪＲ中央線/市ケ谷駅 歩7分都営大江戸線/牛込神楽坂駅 歩13分,"(築3年5階建,)",1階,15.7万円,10000円,15.7万円,-,1LDK,31.36m2
26114,パークアクシス市谷左内町,東京都新宿区市谷左内町,都営新宿線/市ケ谷駅 歩6分ＪＲ中央線/市ケ谷駅 歩7分都営大江戸線/牛込神楽坂駅 歩13分,"(築3年5階建,)",1階,18.8万円,10000円,18.8万円,-,1LDK,36.91m2
26115,パークアクシス市谷左内町,東京都新宿区市谷左内町,都営新宿線/市ケ谷駅 歩6分ＪＲ中央線/市ケ谷駅 歩7分都営大江戸線/牛込神楽坂駅 歩13分,"(築3年5階建,)",3階,21.5万円,10000円,21.5万円,-,1LDK,41.8m2
